### Packages

In [1]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import seaborn as sns
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

### Loading the data

In [2]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2018-01-29", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True, treads = True)

[*********************100%***********************]  4 of 4 completed


In [3]:
df_comp = raw_data.copy()

In [4]:
df_comp.head()

^GSPC                                                   \
                  Open        High         Low       Close       Volume   
Date                                                                      
1994-01-07  467.089996  470.260010  467.029999  469.899994  324920000.0   
1994-01-10  469.899994  475.269989  469.549988  475.269989  319490000.0   
1994-01-11  475.269989  475.279999  473.269989  474.130005  305490000.0   
1994-01-12  474.130005  475.059998  472.140015  474.170013  310690000.0   
1994-01-13  474.170013  474.170013  471.799988  472.470001  277970000.0   

                 ^GDAXI                                                \
                   Open         High          Low        Close Volume   
Date                                                                    
1994-01-07  2218.959961  2227.639893  2201.820068  2224.949951    0.0   
1994-01-10  2231.840088  2238.010010  2222.000000  2225.000000    0.0   
1994-01-11  2225.429932  2235.610107  2225.179932  2228.100098    0.0   
1994-01-12  2227.120117  2227.790039  2182.060059  2182.060059    0.0   
1994-01-13  2171.500000  2183.709961  2134.100098  2142.370117    0.0   

                   ^N225                                                   \
                    Open          High           Low         Close Volume   
Date                                                                        
1994-01-07  17842.980469  18131.410156  17787.480469  18124.009766    0.0   
1994-01-10  18186.519531  18567.060547  18186.519531  18443.439453    0.0   
1994-01-11  18481.849609  18671.669922  18373.039062  18485.250000    0.0   
1994-01-12  18447.339844  18807.080078  18301.929688  18793.880859    0.0   
1994-01-13  18770.380859  18823.380859  18548.750000  18577.259766    0.0   

                  ^FTSE                                                
                   Open         High          Low        Close Volume  
Date                                                                   
1994-01-07  3401.399902  3446.800049  3398.699951  3446.000000    0.0  
1994-01-10  3465.699951  3468.100098  3430.000000  3440.600098    0.0  
1994-01-11  3442.500000  3442.500000  3413.500000  3413.800049    0.0  
1994-01-12  3394.800049  3402.399902  3372.000000  3372.000000    0.0  
1994-01-13  3380.699951  3383.300049  3356.899902  3360.000000    0.0

In [5]:
df_comp.describe()

^GSPC                                                       \
              Open         High          Low        Close        Volume   
count  6057.000000  6057.000000  6057.000000  6057.000000  6.057000e+03   
mean   1287.508251  1295.148670  1279.399984  1287.845508  2.408817e+09   
std     487.423209   488.426764   486.358896   487.549637  1.720534e+09   
min     439.140015   444.720001   435.859985   438.920013  1.499000e+07   
25%     990.469971  1001.789978   981.289978   990.640015  8.959000e+08   
50%    1231.930054  1240.400024  1224.030029  1233.000000  2.121580e+09   
75%    1460.250000  1469.540039  1451.300049  1460.250000  3.648890e+09   
max    2847.479980  2872.870117  2846.179932  2872.870117  1.145623e+10   

             ^GDAXI                                                          \
               Open          High           Low         Close        Volume   
count   6093.000000   6093.000000   6093.000000   6093.000000  6.093000e+03   
mean    6082.705807   6129.330871   6031.415089   6082.141238  8.194499e+07   
std     2748.218630   2760.935598   2735.105180   2748.336521  6.463291e+07   
min     1903.760010   1921.300049   1891.709961   1911.699951  0.000000e+00   
25%     4075.850098   4122.930176   4034.280029   4073.350098  3.024580e+07   
50%     5769.560059   5813.229980   5708.790039   5774.259766  8.552390e+07   
75%     7438.259766   7492.759766   7391.060059   7442.959961  1.170728e+08   
max    13577.139648  13596.889648  13517.809570  13559.599609  5.101956e+08   

              ^N225                                                           \
               Open          High           Low         Close         Volume   
count   5910.000000   5910.000000   5910.000000   5910.000000    5910.000000   
mean   14612.611644  14712.657954  14504.083305  14608.063595   83243.908629   
std     4044.497448   4062.441869   4024.529836   4044.953206   74236.991257   
min     7059.770020   7100.770020   6994.899902   7054.979980       0.000000   
25%    10745.137451  10807.017578  10650.764893  10722.667480       0.000000   
50%    15041.494629  15151.629883  14897.060059  15053.099609   92850.000000   
75%    17844.817871  17961.637207  17698.189941  17860.560059  136500.000000   
max    24078.929688  24129.339844  23917.140625  24124.150391  595200.000000   

             ^FTSE                                                       
              Open         High          Low        Close        Volume  
count  6078.000000  6078.000000  6078.000000  6078.000000  6.078000e+03  
mean   5414.530354  5450.222161  5377.636032  5414.868475  8.986947e+08  
std    1149.347126  1154.006094  1144.946162  1149.462905  6.337472e+08  
min    2844.699951  2902.199951  2844.699951  2876.600098  0.000000e+00  
25%    4469.125122  4502.250122  4442.899902  4469.125122  5.214724e+08  
50%    5656.150146  5698.699951  5605.600098  5657.399902  8.549538e+08  
75%    6300.525024  6347.174805  6258.450073  6302.274902  1.362576e+09  
max    7778.600098  7792.600098  7763.399902  7778.600098  4.420947e+09

In [6]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [7]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

### Creating Returns

In [8]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

### Splitting the Data

In [9]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

### Fitting a Model

In [24]:
pip install --user pmdarima

  Using cached https://files.pythonhosted.org/packages/e5/bf/44888cf13a4ab8c0c3c5c7cd09347f9c27e8e4dc33696a199eb18dacadd3/pmdarima-1.6.1-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/86/3c/d118973b8d85eeba14b3a306a5a06f920dd92878fa57c264dbe70b83a197/statsmodels-0.11.1-cp37-none-win_amd64.whl
Note: you may need to restart the kernel to use updated packages.


In [10]:
from pmdarima.arima import auto_arima

In [11]:
model_auto = auto_arima(df.ret_ftse[1:])

In [12]:
model_auto

ARIMA(order=(4, 0, 5))

In [13]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5019
Model:               SARIMAX(4, 0, 5)   Log Likelihood               -7882.658
Date:                Fri, 29 May 2020   AIC                          15787.316
Time:                        07:15:41   BIC                          15859.047
Sample:                             0   HQIC                         15812.452
                               - 5019                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0309      0.025      1.246      0.213      -0.018       0.080
ar.L1          0.0135      0.082      0.165      0.869      -0.147       0.174
ar.L2         -0.6690      0.077     -8.645      0.000      -0.821      -0.517
ar.L3         -0.1616      0.072     -2.260      0.024      -0.302      -0.021
ar.L4          0.1898      0.074      2.553      0.011       0.044       0.335
ma.L1         -0.0384      0.081     -0.471      0.637      -0.198       0.121
ma.L2          0.6205      0.078      7.933      0.000       0.467       0.774
ma.L3          0.0592      0.069      0.858      0.391      -0.076       0.194
ma.L4         -0.1836      0.073     -2.510      0.012      -0.327      -0.040
ma.L5         -0.1063      0.010    -11.130      0.000      -0.125      -0.088
sigma2         1.3541      0.015     91.128      0.000       1.325       1.383
===================================================================================
Ljung-Box (Q):                       67.77   Jarque-Bera (JB):              6360.07
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               2.00   Skew:                            -0.19
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.50
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Important Arguments

In [14]:
model_auto = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 40, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample = int(len(df_comp)*0.2))


# !!! Important Note: In pdmarima v1.5.2, out_of_sample_size is replaced with out_of_sample, so make sure to use the latter!


# exogenous -> outside factors (e.g other time series)
# m -> seasonal cycle length
# max_order -> maximum amount of variables to be used in the regression (p + q)
# max_p -> maximum AR components
# max_q -> maximum MA components
# max_d -> maximum Integrations
# maxiter -> maximum iterations we're giving the model to converge the coefficients (becomes harder as the order increases)
# alpha -> level of significance, default is 5%, which we should be using most of the time
# n_jobs -> how many models to fit at a time (-1 indicates "as many as possible")
# trend -> "ct" usually
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' 
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple -> validates the model selection (pass the entire dataset, and set 20% to be the out_of_sample_size)

In [15]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                          SARIMAX Results                                           
====================================================================================================
Dep. Variable:                                            y   No. Observations:                 6275
Model:             SARIMAX(0, 0, 1)x(0, 0, [1, 2, 3, 4], 5)   Log Likelihood               -6333.370
Date:                                      Fri, 29 May 2020   AIC                          12688.740
Time:                                              07:20:13   BIC                          12762.928
Sample:                                          01-11-1994   HQIC                         12714.445
                                               - 01-29-2018                                         
Covariance Type:                                        opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0027      0.014     -0.197      0.844      -0.029       0.024
drift      -1.636e-06   4.19e-06     -0.391      0.696   -9.84e-06    6.57e-06
ret_spx        0.0959      0.006     17.063      0.000       0.085       0.107
ret_dax        0.5581      0.005    114.065      0.000       0.549       0.568
ret_nikkei     0.0703      0.004     16.714      0.000       0.062       0.079
ma.L1         -0.1061      0.007    -14.611      0.000      -0.120      -0.092
ma.S.L5       -0.0283      0.008     -3.357      0.001      -0.045      -0.012
ma.S.L10      -0.0497      0.009     -5.440      0.000      -0.068      -0.032
ma.S.L15      -0.0237      0.009     -2.715      0.007      -0.041      -0.007
ma.S.L20      -0.0242      0.009     -2.721      0.007      -0.042      -0.007
sigma2         0.4424      0.004    101.273      0.000       0.434       0.451
===================================================================================
Ljung-Box (Q):                       75.64   Jarque-Bera (JB):             14890.40
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.54   Skew:                             0.24
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.53
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""